In [ ]:
import os
import json
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
matplotlib.rcParams['mathtext.fontset'] = 'cm'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
matplotlib.rcParams["font.size"] = 30

In [ ]:
def load_data(filename):
    with open(filename, "r") as file:
        data = pd.DataFrame(json.load(file))
    data["total_dry_time"]        = data["dry_run_time"]          + data["creation_time"] + data["import_time"]
    data["total_simulation_time"] = data["simulation_times_mean"] + data["creation_time"] + data["import_time"]
    return data

In [ ]:
def plot_scaling(quantity, precision):

    # Load data
    cpu = load_data("./qibo_scaling_cpu.dat")
    gpu = load_data("./qibo_scaling_gpu.dat")

    # Settings
    circuit   = "qft"
    cpu_backends = ["numpy", "tensorflow", "qibotf", "qibojit"]
    gpu_backends = ["tensorflow", "qibotf", "qibojit"]

    # Prepare GPU data
    condition = (gpu["circuit"] == circuit) & (gpu["precision"] == precision)
    data = {f"{k} GPU": gpu[(gpu["library_options"] == f"backend={k}") & condition] for k in gpu_backends}

    # Prepare CPU data
    condition = (cpu["circuit"] == circuit) & (cpu["precision"] == precision)
    data.update({k: cpu[(cpu["library_options"] == f"backend={k}") & condition] for k in cpu_backends})

    # Plot data
    cp  = sns.color_palette("Paired")
    bcp = sns.color_palette("Blues", 3)
    plt.figure(figsize=(14, 8))
    plt.semilogy(data["numpy"]["nqubits"], data["numpy"][quantity], marker="^", markersize=10,
                 color=cp[0], linewidth=3.0, label="numpy")
    plt.semilogy(data["tensorflow"]["nqubits"], data["tensorflow"][quantity], marker="v", markersize=10,
                 color=cp[1], linewidth=3.0, label="tensorflow cpu")
    plt.semilogy(data["qibotf"]["nqubits"], data["qibotf"][quantity], marker="v", markersize=10,
                 color=cp[2], linewidth=3.0, label="qibotf cpu")
    plt.semilogy(data["qibojit"]["nqubits"], data["qibojit"][quantity],
                 color=cp[3], linewidth=3.0, label="qibojit cpu")
    plt.semilogy(data["tensorflow GPU"]["nqubits"], data["tensorflow GPU"][quantity], marker="v", markersize=10,
                 color=bcp[0], linewidth=3.0, label="tensorflow gpu")
    plt.semilogy(data["qibotf GPU"]["nqubits"], data["qibotf GPU"][quantity], marker="v", markersize=10,
                 color=bcp[1], linewidth=3.0, label="qibotf gpu")
    plt.semilogy(data["qibojit GPU"]["nqubits"], data["qibojit GPU"][quantity],
                 color=bcp[2], linewidth=3.0, label="qibojit gpu")

    plt.title(f"{circuit}, {precision} precision")
    plt.xlabel("Number of qubits")
    if quantity == "total_dry_time":
        plt.ylabel("Total dry run time (sec)")
    elif quantity == "total_simulation_time":
        plt.ylabel("Total simulation time (sec)")
    #plt.legend(bbox_to_anchor=(1, 1))
    plt.legend(loc="upper left")
    plt.savefig(f"qibo_scaling_{quantity}_{precision}.pdf", bbox_inches="tight")

In [ ]:
plot_scaling("total_simulation_time", "double")

In [ ]:
plot_scaling("total_dry_time", "double")

In [ ]:
plot_scaling("total_simulation_time", "single")

In [ ]:
plot_scaling("total_dry_time", "single")